In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [2]:
import pickle

with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)
with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [3]:
import numpy as np 
import scipy.sparse as sps
profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)

#profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
profile_length, profile_length.shape

(array([36, 28,  3, ...,  5, 20, 56], dtype=int32), (12638,))

In [9]:
# PROFILE LENGTH BASED GROUPING
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

MAP_recommender_per_group = {}
cutoff = 10
threshold = 30

# Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group

users_in_group_flag = profile_length < threshold
users_in_group = np.arange(len(profile_length))[users_in_group_flag]

users_in_group_p_len = profile_length[users_in_group]

# Select users not in the current group
users_not_in_group_flag = np.isin(np.arange(len(profile_length)), users_in_group, invert=True)
users_not_in_group = np.arange(len(profile_length))[users_not_in_group_flag]

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 3936 Users


In [8]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = SLIMElasticNetRecommender(URM_train_validation)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)#, workers=7)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train_validation)
rp3beta.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
## USER KNN
userknn = UserKNNCFRecommender(URM_train_validation)
userknn.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train_validation)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
## EASE_R
ease_r = EASE_R_Recommender(URM_train_validation)
ease_r.fit(topK=24, l2_norm=37.54323189430143)

model = LinearHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 4.51 min. Items per second: 82.14
RP3betaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7454.18 column/sec. Elapsed time 2.98 sec
UserKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 10183.31 column/sec. Elapsed time 1.24 sec
ItemKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 10183.49 column/sec. Elapsed time 2.18 sec
EASE_R_Recommender: URM Detected 205 ( 1.6%) users with

In [10]:
knn = TwoScoresHybridRecommender(URM_train_validation, userknn, itemknn)
knn.fit(alpha=0.022195783788315104)

TwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [6]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
multvae = MultVAERecommender(URM_train_validation)
multvae.load_model(folder_path="slim_models/", file_name="MultVAE_train_validation_300")

MultVAERecommender: URM Detected 205 ( 1.6%) users with no interactions.
MultVAERecommender: URM Detected 109 ( 0.5%) items with no interactions.
MultVAERecommender: Loading model from file 'slim_models/MultVAE_train_validation_300'


2024-01-03 00:00:03.635938: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-03 00:00:03.635964: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-03 00:00:03.635969: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-03 00:00:03.635999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-03 00:00:03.636016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-01-03 00:00:03.734153: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-01-03 00:00:03.737097: I tensorflow/core/gra

INFO:tensorflow:Restoring parameters from slim_models/MultVAE_train_validation_300/.session/session
MultVAERecommender: Loading complete


In [7]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
ials = IALSRecommender(URM_train_validation)
ials.load_model(folder_path="slim_models/", file_name="IALS_train_validation")

IALSRecommender: URM Detected 205 ( 1.6%) users with no interactions.
IALSRecommender: URM Detected 109 ( 0.5%) items with no interactions.
IALSRecommender: Loading model from file 'slim_models/IALS_train_validation'
IALSRecommender: Loading complete


In [17]:
hybrid = LinearHybridRecommender(URM_train_validation, [slim, rp3beta, knn, ease_r])
hybrid.fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])
hybrid_recommender = LinearHybridRecommender(URM_train_validation, [hybrid, ials])

SLIMElasticNetRP3betaTwoScoresHybridEASE_R_HybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridEASE_R_HybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridEASE_R_HybridIALSHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridEASE_R_HybridIALSHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [19]:
import optuna as op
def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    beta = trial.suggest_float('beta', 0, 1)
    
    # Create hybrid recommender with the suggested weights
    hybrid_recommender.fit(alphas=[alpha,beta])
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [23]:
study_name = "hybrid-IALS-2-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=50)

[I 2024-01-03 00:42:42,224] Using an existing study with name 'hybrid-IALS-2-study' instead of creating a new one.


EvaluatorHoldout: Processed 6499 (100.0%) in 6.43 sec. Users per second: 1011


[I 2024-01-03 00:42:48,724] Trial 100 finished with value: 0.027760662812593755 and parameters: {'alpha': 0.9311482355816865, 'beta': 0.11140454328425922}. Best is trial 91 with value: 0.027817093960731346.


EvaluatorHoldout: Processed 6499 (100.0%) in 5.46 sec. Users per second: 1190


[I 2024-01-03 00:42:54,227] Trial 101 finished with value: 0.0277887684796439 and parameters: {'alpha': 0.8967311811063379, 'beta': 0.041641596201290085}. Best is trial 91 with value: 0.027817093960731346.


...

[I 2024-01-03 00:46:36,406] Trial 147 finished with value: 0.027763526501024472 and parameters: {'alpha': 0.9232927978143761, 'beta': 0.05548637684887968}. Best is trial 91 with value: 0.027817093960731346.


EvaluatorHoldout: Processed 6499 (100.0%) in 4.86 sec. Users per second: 1339


[I 2024-01-03 00:46:41,309] Trial 148 finished with value: 0.02747506942460001 and parameters: {'alpha': 0.999845877915388, 'beta': 0.46586141992964636}. Best is trial 91 with value: 0.027817093960731346.


EvaluatorHoldout: Processed 6499 (100.0%) in 4.87 sec. Users per second: 1335


[I 2024-01-03 00:46:46,219] Trial 149 finished with value: 0.02775440421847548 and parameters: {'alpha': 0.8781985889896188, 'beta': 0.02694206287205283}. Best is trial 91 with value: 0.027817093960731346.


In [16]:
study.best_params

{'alpha': 0.8857667100747117,
 'beta': 0.6379807942443021,
 'gamma': 0.0690929184825888,
 'delta': 0.11953478623354052}